In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Input, Dense, LSTM,Conv2D,Conv1D,Flatten,MaxPooling2D,GlobalAveragePooling2D,Dropout,Bidirectional,Attention
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import tensorflow as tf
import keras

In [ ]:
pip install shap

In [ ]:
import shap
import numpy as np


In [ ]:

# Display the version of each library
print("pandas version:", pd.__version__)
print("matplotlib version:", plt.matplotlib.__version__)
print("seaborn version:", sns.__version__)
print("numpy version:", np.__version__)
print("scikit-learn version:", StandardScaler().fit_transform([[0]])[0, 0])  # sklearn version
print("tensorflow version:", tf.__version__)  # assuming 'tf' is your TensorFlow import
print("keras version:", keras.__version__)

In [ ]:
warnings.filterwarnings(action="ignore")

In [ ]:
df = pd.read_excel("/kaggle/input/algo-banknifty/Test Data Set  Requirements.xlsx",sheet_name="Data")

In [ ]:
sta = df[df.strike == 1]

In [ ]:
sta["spot_closing"] = df[df.strike == 0].open.to_list()
sta["diff_closing_opening"] = sta.close.shift() - sta.open
sta["oi_change"] = np.log(sta.oi).pct_change()
sta["log_Target_price"] = np.log(sta.close)
sta["log_Target_return"] = sta.log_Target_price.pct_change()
sta["Target"] = sta["log_Target_return"].apply(lambda x: 0 if x > 0 else 1)
sta.fillna(0,inplace=True)
sta.drop(columns=["type","strike"],inplace=True)
sta.head()

In [ ]:
def option_value(data, refer, _type_, shift,which):
    ls = []
    df = refer
    if which == "price":
        which = "open"
    for i in data.date:
        element = df[(df.date == i) & (df["type"]==_type_) &
                (df["strike"] == int(data.loc[data[data.date == i].index,"strike_price"].values[0] + shift))][f"{which}"].values[0]
        
        ls.append(element)
    return ls

In [ ]:
sta["strike_price"] = round(sta["spot_closing"]/100)*100
sta["atm_ce_price"] = option_value(sta, df, "CE", shift=0,which="price") 
sta["atm_pe_price"] = option_value(sta, df, "PE", shift=0,which="price") 
sta["atm_ce_oi"] = option_value(sta, df, "CE", shift=0,which="oi") 
sta["atm_pe_oi"] = option_value(sta, df, "PE", shift=0,which="oi")

In [ ]:
sta["1itm_ce_price"] = option_value(sta,df,"CE",shift=(-100),which="price")
sta["1itm_pe_price"] = option_value(sta,df,"PE",shift=(-100),which="price")
sta["1itm_ce_oi"] = option_value(sta,df,"CE",shift=(-100),which="oi")
sta["1itm_pe_oi"] = option_value(sta,df,"PE",shift=(-100),which="oi") 
sta["2itm_ce_price"] = option_value(sta,df,"CE",shift=(-200),which="price") 
sta["2itm_pe_price"] = option_value(sta,df,"PE",shift=(-200),which="price")
sta["2itm_ce_oi"] = option_value(sta,df,"CE",shift=(-200),which="oi") 
sta["2itm_pe_oi"] = option_value(sta,df,"PE",shift=(-200),which="oi") 
sta["3itm_ce_price"] = option_value(sta,df,"CE",shift=(-300),which="price") 
sta["3itm_pe_price"] = option_value(sta,df,"PE",shift=(-300),which="price")
sta["3itm_ce_oi"] = option_value(sta,df,"CE",shift=(-300),which="oi")
sta["3itm_pe_oi"] = option_value(sta,df,"PE",shift=(-300),which="oi") 


sta["1otm_ce_price"] = option_value(sta,df,"CE",shift=(100),which="price")
sta["1otm_pe_price"] = option_value(sta,df,"PE",shift=(100),which="price")
sta["1otm_ce_oi"] = option_value(sta,df,"CE",shift=(100),which="oi")
sta["1otm_pe_oi"] = option_value(sta,df,"PE",shift=(100),which="oi") 
sta["2otm_ce_price"] = option_value(sta,df,"CE",shift=(200),which="price") 
sta["2otm_pe_price"] = option_value(sta,df,"PE",shift=(200),which="price")
sta["2otm_ce_oi"] = option_value(sta,df,"CE",shift=(200),which="oi") 
sta["2otm_pe_oi"] = option_value(sta,df,"PE",shift=(200),which="oi") 
sta["3otm_ce_price"] = option_value(sta,df,"CE",shift=(300),which="price") 
sta["3otm_pe_price"] = option_value(sta,df,"PE",shift=(300),which="price")
sta["3otm_ce_oi"] = option_value(sta,df,"CE",shift=(300),which="oi")
sta["3otm_pe_oi"] = option_value(sta,df,"PE",shift=(300),which="oi") 

In [ ]:
sta["consecutive_ce_oi"] = sta['atm_ce_oi'] + sta['1itm_ce_oi'] + sta['2itm_ce_oi'] + sta['3itm_ce_oi'] + sta['1otm_ce_oi'] +sta['1otm_ce_oi'] +sta['3otm_ce_oi'] 

In [ ]:
sta["consecutive_pe_oi"] = sta['atm_pe_oi'] + sta['1itm_pe_oi'] + sta['2itm_pe_oi'] + sta['3itm_pe_oi'] + sta['1otm_pe_oi'] +sta['1otm_pe_oi'] +sta['3otm_pe_oi'] 

In [ ]:
sta["ce_oi_change"] = np.log(sta["consecutive_ce_oi"]).pct_change()
sta["pe_oi_change"] = np.log(sta["consecutive_pe_oi"]).pct_change()

In [ ]:
sta.columns

In [ ]:
data = sta[["Target","date", 'open','atm_ce_price','volume', 'oi',
       'spot_closing', 'diff_closing_opening', 'oi_change',
       'atm_pe_price', 'atm_ce_oi', 'atm_pe_oi', '1itm_ce_price',
       '1itm_pe_price', '1itm_ce_oi', '1itm_pe_oi', '2itm_ce_price',
       '2itm_pe_price', '2itm_ce_oi', '2itm_pe_oi', '3itm_ce_price',
       '3itm_pe_price', '3itm_ce_oi', '3itm_pe_oi', '1otm_ce_price',
       '1otm_pe_price', '1otm_ce_oi', '1otm_pe_oi', '2otm_ce_price',
       '2otm_pe_price', '2otm_ce_oi', '2otm_pe_oi', '3otm_ce_price',
       '3otm_pe_price', '3otm_ce_oi', '3otm_pe_oi', 'consecutive_ce_oi',
       'consecutive_pe_oi', 'ce_oi_change', 'pe_oi_change']]
data = data[data.date > "2023-02-02 09:20:00"]
print(data.shape)
data.head()


In [ ]:
data["log_Target_return"] = data["log_Target_return"]*100

In [ ]:
data.head()

In [ ]:
sta.head()

In [ ]:
data = data.drop(columns=["date","Target"])
corr = data.corr()

# Create a heatmap with annotations
sns.set(style='white')
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(50, 50))
sns.heatmap(corr, mask=mask, cmap='coolwarm', annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
corr_matrix = data.corr()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.98)]
print("No. of Columns to drop",len(to_drop))
to_drop

## model build

In [ ]:
def SlidingWindow(df, w, s):
    X = []
    Y = []
    # Iterate through the rows of the DataFrame in steps of size s
    for i in range(0, len(df)-w, s):
        # Extract a window of width w from the DataFrame, starting at row i
        x = np.array(df.iloc[i:i+w, 1:])
        # Extract the target value for the last row of the window
        y = np.array(df.iloc[i+w-1, 0])
        # Append the window and target value to the X and Y lists, respectively
        X.append(x)
        Y.append(y)

    # Convert the X and Y lists to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [ ]:
X, Y = SlidingWindow(data.drop(columns="date"), 10, 1)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
sc = StandardScaler()
sc.fit(data.drop(columns=["Target","date"]))

In [ ]:
X_sc = sc.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

In [ ]:
# Create an instance of the OneHotEncoder with the 'sparse' parameter set to False
enc = OneHotEncoder(sparse=False)

# Fit the OneHotEncoder to the target values Y, which must be reshaped to have a single column
enc.fit(Y.reshape(-1, 1))

# Transform the target values Y to one-hot encoded form, by applying the OneHotEncoder to the reshaped Y array
Y_enc = enc.transform(Y.reshape(-1, 1))

In [ ]:
len(X)

In [ ]:
cutoff = 320
x_train = X_sc[:cutoff]
y_train = Y_enc[:cutoff]
x_test = X_sc[cutoff:]
y_test = Y_enc[cutoff:]

In [ ]:
Y[:5]

In [ ]:
y_train[:5]

In [ ]:
def NN_model(x_train, y_train):
    # Define input layer with the correct input shape
    inputs = Input(shape=(x_train.shape[1]))

    # Define hidden layers with 100 nodes and selu activation function
    hidden_layer = Dense(100, activation='selu')(inputs)
    hidden_layer = Dense(100, activation='selu')(hidden_layer)

    # Define output layer with softmax activation function
    outputs = Dense(y_train.shape[1], activation='softmax')(hidden_layer)

    # Define the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model with categorical cross-entropy loss function and Adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Print the summary of the model
    print(model.summary())
    return model


In [ ]:

def LSTM_model(X_train,y_train):
    # Define input layer
    input_layer = Input(shape=(X_train.shape[1],X_train.shape[2]))

    # Define encoder layers
    encoded = Bidirectional(LSTM(128,activation="tanh",return_sequences=True))(input_layer)
    encoded = LSTM(128, activation="tanh")(encoded)
    
    # Define decoder layers
    decoded = Dense(300, activation='relu')(encoded)
    decoded = Dropout(0.5)(decoded)
    decoded = Dense(y_train.shape[1], activation='sigmoid')(decoded)

    # Define LSTM model
    lstm_model = Model(inputs=input_layer, outputs=decoded)
    # Compile LSTM  model
    lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(lstm_model.summary())
    return lstm_model

In [ ]:
def Simple_LSTM_model(X_train, y_train):
    # Define input layer
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

    # Single LSTM layer
    lstm_layer = LSTM(128, activation="tanh")(input_layer)

    # Dense layer for classification
    output_layer = Dense(y_train.shape[1], activation='sigmoid')(lstm_layer)  # 'sigmoid' for binary classification

    # Define LSTM model
    lstm_model = Model(inputs=input_layer, outputs=output_layer)

    # Compile LSTM model
    lstm_model.compile(loss='binary_crossentropy',  # 'binary_crossentropy' for binary classification
                       optimizer='adam',
                       metrics=['accuracy'])

    return lstm_model

In [ ]:
def CNN_model(X_train, y_train):
    # Define input layer
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

    # Define encoder layers
    encoded = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
    encoded = Conv1D(filters=32, kernel_size=3, activation='relu') (encoded)

    encoded = Flatten()(encoded)

    decoded = Dense(units=128, activation='relu')(encoded)
    decoded = Dropout(rate=0.5)(decoded)

    decoded = Dense(units=y_train.shape[1], activation='softmax')(decoded)

    # Define autoencoder model
    cnn_model = Model(inputs=input_layer, outputs=decoded)

    # Compile autoencoder model with additional regularization
    cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return cnn_model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)
# Create model
model1 = LSTM_model(x_train,y_train)
# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model1.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

# Plot the training history for loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)
# Create model
model3 = Simple_LSTM_model(x_train,y_train)
# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model3.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

# Plot the training history for loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)
# Create model
model2 = CNN_model(x_train,y_train)
# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model2.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

# Plot the training history for loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
X = data.drop(columns=["date","Target"])
Y = np.array(data["Target"])

sc = StandardScaler()
sc.fit(data.drop(columns=["Target","date"]))
X_sc = sc.transform(X)

# Create an instance of the OneHotEncoder with the 'sparse' parameter set to False
enc = OneHotEncoder(sparse=False)

# Fit the OneHotEncoder to the target values Y, which must be reshaped to have a single column
enc.fit(Y.reshape(-1, 1))

# Transform the target values Y to one-hot encoded form, by applying the OneHotEncoder to the reshaped Y array
Y_enc = enc.transform(Y.reshape(-1, 1))

In [ ]:
print(X_sc.shape)

In [ ]:
cutoff = 250
x_train = X_sc[:cutoff]
y_train = Y_enc[:cutoff]
x_test = X_sc[cutoff:]
y_test = Y_enc[cutoff:]

In [ ]:

early_stop = EarlyStopping(monitor='val_loss', patience=15)
# Create model
model4 = NN_model(x_train,y_train)
# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model4.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

# Plot the training history for loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

# Evaluation of LSTM model for return

In [ ]:
# Assuming you have trained your model and have test data (X_test, y_test)

# # Evaluate the model on the test set
# evaluation = model.evaluate(x_test, y_test)
# # evaluation1 = model1_return.evaluate(x_test, y_test)

# # Print the evaluation metrics
# print("Mean Squared Error (MSE) model2:", evaluation[0])
# print("Mean Absolute Error (MAE) model2:", evaluation[1])

# Print the evaluation metrics
# print("Mean Squared Error (MSE) model1:", evaluation1[0])
# print("Mean Absolute Error (MAE) model1:", evaluation1[1])

In [ ]:
# Predictions on the test set
predictions = model.predict(x_test)

# Plotting actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions)
plt.title('Actual vs Predicted Stock Returns')
plt.xlabel('Actual Returns')
plt.ylabel('Predicted Returns')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
y_pred = model.predict(x_test)
# mean_absolute_error(y_pred,y_test)

In [ ]:
test = pd.DataFrame(y_pred,columns=["predicted"])
test["date"] = data.date[260:].to_list()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.lineplot(x=data["date"],y=data["log_Target_return"])
sns.lineplot(x=test["date"],y=test["predicted"])
plt.show()

# Evaluation of LSTM model


In [ ]:
y_pred = model1.predict(x_test, verbose=0)
y_pred = np.where(y_pred>0.4,0,1)
for i in range(10):
    print(y_pred[i],y_test[i])

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    f, ax = plt.subplots(figsize=(5, 5))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, cmap='Blues',ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    plt.tight_layout()
    plt.show()

y_pred = enc.inverse_transform(model1.predict(x_test, verbose=0))
y_true = enc.inverse_transform(y_test)
    
# Plot the confusion matrix and print the f1 score for each algorithm
plot_confusion_matrix(y_true, y_pred, 'LSTM Confusion Matrix')
print("LSTM accuracy score:", accuracy_score(y_true, y_pred))

### classification report

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y_true, y_pred))
print(f"AUC:{roc_auc_score(y_true, y_pred)}")

### Featrure Importance

In [ ]:
weights = model1.get_weights()
w = pd.DataFrame(weights[0],index=data.drop(columns=["Target","date"]).columns.to_list())
w['sum_all_columns'] = w.sum(axis=1)
w.head(10)["sum_all_columns"].plot(kind="barh")

In [ ]:
w_sorted = w.sort_values(by="sum_all_columns")

# Create subplots
fig, ax = plt.subplots(figsize=(14, 6), ncols=2)

# Plot the top 10
w_sorted.tail(10)["sum_all_columns"].plot(kind="barh", ax=ax[0])
ax[0].set_title('10 most important feature')

# Plot the bottom 10
w_sorted.head(10)["sum_all_columns"].plot(kind="barh", ax=ax[1])
ax[1].set_title('10 Least important feature')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

y_ticks = [0, 1]
y_labels = ['Buy', 'Sell']

# Plot the first subplot (top)
axs[0].plot(y_true, color='black', label='Test')
axs[0].set_yticks(y_ticks)
axs[0].set_yticklabels(y_labels)
axs[0].legend()

# Plot the second subplot (bottom)
axs[1].plot(y_pred, color='green', label='Pred')
axs[1].set_yticks(y_ticks)
axs[1].set_yticklabels(y_labels)
axs[1].legend()

# Show the plots
plt.show()

# Simple UNidiretional LSTM

In [ ]:
y_pred = enc.inverse_transform(model3.predict(x_test, verbose=0))
y_true = enc.inverse_transform(y_test)
    
# Plot the confusion matrix and print the f1 score for each algorithm
plot_confusion_matrix(y_true, y_pred, 'LSTM Confusion Matrix')
print("LSTM accuracy score:", accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y_true, y_pred))
print(f"AUC:{roc_auc_score(y_true, y_pred)}")

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

y_ticks = [0, 1]
y_labels = ['Buy', 'Sell']

# Plot the first subplot (top)
axs[0].plot(y_true, color='black', label='Test')
axs[0].set_yticks(y_ticks)
axs[0].set_yticklabels(y_labels)
axs[0].legend()

# Plot the second subplot (bottom)
axs[1].plot(y_pred, color='green', label='Pred')
axs[1].set_yticks(y_ticks)
axs[1].set_yticklabels(y_labels)
axs[1].legend()

# Show the plots
plt.show()

# CNN

In [ ]:
y_pred = enc.inverse_transform(model2.predict(x_test, verbose=0))
y_true = enc.inverse_transform(y_test)
    
# Plot the confusion matrix and print the f1 score for each algorithm
plot_confusion_matrix(y_true, y_pred, 'CNN Confusion Matrix')
print("CNN accuracy score:", accuracy_score(y_true, y_pred))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

y_ticks = [0, 1]
y_labels = ['Buy', 'Sell']

# Plot the first subplot (top)
axs[0].plot(y_true, color='black', label='Test')
axs[0].set_yticks(y_ticks)
axs[0].set_yticklabels(y_labels)
axs[0].legend()

# Plot the second subplot (bottom)
axs[1].plot(y_pred, color='green', label='Pred')
axs[1].set_yticks(y_ticks)
axs[1].set_yticklabels(y_labels)
axs[1].legend()

# Show the plots
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

y_ticks = [0, 1]
y_labels = ['Buy', 'Sell']

# Plot the first subplot (top)
axs[0].plot(y_true, color='black', label='Test')
axs[0].set_yticks(y_ticks)
axs[0].set_yticklabels(y_labels)
axs[0].legend()

# Plot the second subplot (bottom)
axs[1].plot(y_pred, color='green', label='Pred')
axs[1].set_yticks(y_ticks)
axs[1].set_yticklabels(y_labels)
axs[1].legend()

# Show the plots
plt.show()

# BackTest

In [ ]:
def backtest(y_pred, lot_size, nn=False):
    predicted =pd.DataFrame(y_pred,columns=["predicted"])
    if nn:
        cutoff = 250
    else:
        cutoff = 330
    predicted["date"] = data.date[cutoff:].to_list()
    predicted["Action"]=predicted["predicted"].apply(lambda x: "Sell" if x==1 else "Buy")
    predicted["price"] = sta[sta.date > "2023-02-02 09:20:00"].close[cutoff:].to_list()
    predicted["lot amount"] = predicted["price"] * 1.5 * lot_size
    predicted["new_pred"] = predicted["predicted"].apply(lambda x:(-1) if x == 0 else 1)
    predicted["diff"] = predicted["new_pred"] - predicted["new_pred"].shift(1)
    predicted.fillna(0,inplace=True)
    predicted["predicted1"] = predicted.apply(lambda row: 2 if row["diff"] != 0 else row["predicted"], axis=1)
    predicted.at[len(predicted)-1,"predicted1"] = 2
    t1 = predicted[predicted.predicted1 != 2][["date","Action","price","lot amount"]]
    t1.columns = ["Entry Time","Trade",'Entry Price',"Entry Amount"]
    t1["closed_position"] = "Yes"
    t2 = predicted[predicted.predicted1 == 2][["date","price","lot amount"]]
    t2.columns = ["Exit Time", "Exit Price","Exit Amount"]
    exit_time = []
    count = 0
    for i_1, d_1 in t2.iterrows():
        d1 = d_1["Exit Time"]
        for i_2,d_2 in t1[count:].iterrows():
            if d1 > d_2["Entry Time"]:
                exit_time.append(d1)
                count += 1
            else:
                break

    t1["Exit Time"] = exit_time
    backtest = pd.merge(t1,t2,'inner')
    backtest["profit/loss"] = np.where(backtest["Trade"] == "Buy",
                                       -1 * (backtest["Entry Amount"] - backtest["Exit Amount"]),
                                       backtest["Entry Amount"] - backtest["Exit Amount"])
    print(f"Total profit from this statergy is {backtest['profit/loss'].cumsum()[len(backtest)-1] - len(backtest) * 40}.")
    return backtest

In [ ]:
CNN_y_pred = enc.inverse_transform(model2.predict(x_test, verbose=0))
CNN_backtest = backtest(CNN_y_pred, 1)

In [ ]:
CNN_backtest[CNN_backtest.Trade == "Sell"]['profit/loss']

In [ ]:
LSTM_y_pred = enc.inverse_transform(model1.predict(x_test, verbose=0))
LSTM_backtest = backtest(LSTM_y_pred, 1)

In [ ]:
LSTM_backtest

In [ ]:
LSTM_backtest.to_csv("LSTM_backtest.csv")

In [ ]:
LSTM_backtest[LSTM_backtest.Trade == "Buy"]['profit/loss'].cumsum()

In [ ]:
NN_y_pred = enc.inverse_transform(model3.predict(x_test, verbose=0))
NN_backtest = backtest(NN_y_pred, 1, nn=True)

# Shap Explainer
## model1

In [ ]:
x_test[1].flatten().shape

In [ ]:
model1.summary()

In [ ]:
attention_model = tf.keras.Model(inputs=model1.input, outputs=attention)
attention_weights = attention_model.predict(x_test[:1])

# Plot attention weights
plt.imshow(attention_weights[0], cmap='viridis', aspect='auto')
plt.xlabel('Time steps')
plt.ylabel('Input features')
plt.title('Attention Weights')
plt.colorbar()
plt.show()

## model2

In [ ]:
model1.save("model_LSTM.pkl")
model2.save("model_CNN.pkl")